In [45]:
from narupa.app.client import NarupaClient
from narupa.ase.converter import frame_data_to_ase
import numpy as np
import subprocess
import nglview
import shlex

## Run the server

Run the LAMMPS Lammps or OpenMM or anything else. 

In [52]:
process = subprocess.Popen(['./lmp'], stdin=open('in.peptide'))
print('Started LAMMPS server')

Started LAMMPS server


## Start an IMD client

In [53]:
class CustomClient(NarupaClient):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._view = None
    
    @property
    def view(self):
        if self._view is None:
            atoms = frame_data_to_ase(self.first_frame, topology=True, positions=False)
            atoms.set_positions(np.array(self.latest_frame.particle_positions) * 10)
            self._view = nglview.show_ase(atoms)
        return self._view
    
    def _on_frame_received(self, frame_index, frame):
        super()._on_frame_received(frame_index, frame)
        self.view.set_coordinates({0: np.array(self.latest_frame.particle_positions) * 10})

In [54]:
client = CustomClient(address='localhost',trajectory_port=8080, imd_port=8081)


In [55]:
atoms = frame_data_to_ase(client.first_frame, topology=True, positions=False)
atoms.set_positions(np.array(client.latest_frame.particle_positions) * 10)

In [56]:
client.view

NGLWidget()

In [40]:
## Start a sping interaction

In [57]:
from narupa.imd.particle_interaction import ParticleInteraction
interaction = ParticleInteraction("5", "0", position=(0,0,0), particles=(68), scale=0.1, interaction_type="spring")
interaction_id = client.start_interaction()
client.update_interaction(interaction_id, interaction)

In [9]:
client.stop_interaction(interaction_id)

In [51]:
lmp_pid = process.pid
process.terminate()
process.kill()
process.wait()
print("LAMMPS was killed on pid", lmp_pid)

LAMMPS was killed on pid 15242
